In [40]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os

In [32]:
def xarray_to_csv(ds, title, feature, is_chirps):
    """
    Convert an xarray DataArray to a pandas Series.
    """
    if is_chirps:
        data_ds = ds[feature].sel(
            latitude=ds.latitude.sel(latitude=20.9047, method='nearest'),
            longitude=ds.longitude.sel(longitude=37.1592, method='nearest'),
            method='nearest'
        )
    else:
        data_ds = ds[feature].sel(
            lat=ds.lat.sel(lat=20.9047, method='nearest'),
            lon=ds.lon.sel(lon=37.1592, method='nearest'),
            method='nearest'
        )
    data_df = data_ds.to_dataframe().reset_index()
    data_df.to_csv(f"csv/{title.replace(' ', '_')}_37.1592_20.9047.csv", index=False)

In [33]:
links = {"NASA GISS Surface Temperature Analysis Long Term Mean" : ["air.2x2.250.mon.1991-2020.ltm.comb.nc", 'air'],
         "NOAA Extended Reconstructed SST V5 Long Term Mean" : ["sst.mnmean.nc", 'sst'],
         "Global Precipitation Climatology Project (GPCP) Monthly Analysis Product" : ["precip.mon.mean.nc", 'precip'],
         "CHIRPS V3 Monthly Mean" : ["chirps-v3.0.monthly.nc", 'precip'],
         "NOAA Global Surface Temperature Long Term Mean" : ["air.mon.v6.ltm.1991.2020.nc", 'air'],
         "NOAA OI SST V2 High Resolution Dataset" : ["sst.mon.mean.nc", 'sst']}

In [34]:
for title, ls in links.items():
    filename, feature = ls[0], ls[1]
    ds = xr.open_dataset(filename)
    is_chirps = title[:6] == "CHIRPS"
    xarray_to_csv(ds, title, feature, is_chirps)
    ds.close()

/Users/chaiharsha/Documents/CobbLab/CobbLab/env/lib/python3.9/site-packages/xarray/coding/times.py:992: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/chaiharsha/Documents/CobbLab/CobbLab/env/lib/python3.9/site-packages/xarray/core/indexing.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)
/Users/chaiharsha/Documents/CobbLab/CobbLab/env/lib/python3.9/site-packages/xarray/coding/times.py:992: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_c

In [44]:
records = []
for path in sorted(glob.glob("salinity_data/IAP_Salinity_*.nc")):
    ds = xr.open_dataset(path)

    # Get time info
    year = int(ds.attrs.get("StartYear"))
    month = int(ds.attrs.get("StartMonth"))
    timestamp = pd.Timestamp(f"{year}-{month:02d}-01")

    # Extract salinity
    sal_val = ds["salinity"].sel(
        lat=ds.lat.sel(lat=21.0, method="nearest"),
        lon=ds.lon.sel(lon=37.5, method="nearest"),
        depth_std=5.0,
        method="nearest"
    ).item()

    records.append({
        "time": timestamp,
        "lat" : 21.0,
        "lon": 37.5,
        "depth_std": 5.0,
        "salinity": sal_val
    })

    ds.close()

data_df = pd.DataFrame(records).sort_values("time")
data_df.to_csv(f"csv/{title.replace(' ', '_')}_37.5_21.0.csv", index=False)